In [ ]:
# Prob 1Load Libraries
import pandas as pd
from rpy2.robjects import r, pandas2ri
from rpy2.robjects.conversion import localconverter
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
import numpy as np
from statsmodels.tsa.ar_model import AutoReg
# Activate pandas-to-R conversion
pandas2ri.activate() 

In [ ]:
# 1A Import and view Data

# Load the airquality dataset from R
r_data = r('datasets::airquality')

# Convert to pandas DataFrame
with localconverter(pandas2ri.converter):
    airquality_df = pandas2ri.rpy2py(r_data)

# Display the DataFrame
print(airquality_df)
print(airquality_df.describe()) 

wind = airquality_df['Wind']

# Count NA values
na_count = wind.isna().sum().sum()

# Count infinite values
inf_count = np.isinf(wind).sum().sum()
print(f"NA count: {na_count}, 'Infinity count': {inf_count}")

In [ ]:
# Prob 1A
# Wind: Average wind speed in miles per hour at 0700 and 1000 hours at LaGuardia Airport
plt.figure(figsize=(10, 6))
plt.plot(wind, marker='o', linestyle='-')
plt.xlabel('Day')
plt.ylabel('Avg Wind Speed')
plt.title('Average wind speed in miles per hour at 0700 and 1000 hours at LaGuardia Airport')
plt.grid(True)
plt.show()
# The most prevailing characterisitic seams to be either a autocorrelation or seasonality

# Create the ACF plot
plt.figure(figsize=(10, 6))
plot_acf(wind, lags=40)
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.title('Autocorrelation Function (ACF) for Wind')
plt.grid(True)
plt.show()
# It appears that the most prevealing characteristic of the time serries is that is is mean reverting

auto_reg_model = AutoReg(wind, lags=1).fit()
print(auto_reg_model.summary())

# Produce the ACF plot of the residuals
residuals = auto_reg_model.resid
plt.figure(figsize=(10, 6))
plot_acf(residuals, lags=150)
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.title('Autocorrelation Function (ACF) for Residuals')
plt.grid(True)
plt.show()

In [ ]:
# 1B Import and view Data

# Load the airquality dataset from R
r_data2 = r('datasets::austres')

# Display the DataFrame
print(r_data2)

# Create a pandas DataFrame 
index = np.arange(len(r_data2))
australian_res_df = pd.DataFrame({
    't': index,
    'Australian Residents': r_data2
})
australian_res_df.set_index('t', inplace=True)
print(australian_res_df.describe())

# Count NA values
na_count = australian_res_df.isna().sum().sum()

# Count infinite values
inf_count = np.isinf(australian_res_df).sum().sum()
print(f"NA count: {na_count}, 'Infinity count': {inf_count}")


In [ ]:
# Prob 1B
# Wind: Average wind speed in miles per hour at 0700 and 1000 hours at LaGuardia Airport
plt.figure(figsize=(10, 6))
plt.plot(australian_res_df['Australian Residents'], marker='o', linestyle='-')
plt.xlabel('T')
plt.ylabel('Average Australian Residents')
plt.title('Average Australian Residents from 1971 to 1994')
plt.grid(True)
plt.show()
# The most prevailing characterisitic seams to a strong trend (with some autocorrelation)

# Create the ACF plot
plt.figure(figsize=(10, 6))
plot_acf(australian_res_df['Australian Residents'], lags=79)
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.title('Autocorrelation Function (ACF) for Australian Residents')
plt.grid(True)
plt.show()
# The autocorrelation plot shows a slow decay in the autocorrelation, indicating that the data could be a random walk

auto_reg_model_australian_res = AutoReg(australian_res_df['Australian Residents'], lags=1).fit()
print(auto_reg_model_australian_res.summary())

# Produce the ACF plot of the residuals
residuals_australian_res = auto_reg_model_australian_res.resid
plt.figure(figsize=(10, 6))
plot_acf(residuals_australian_res, lags=79)
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.title('Autocorrelation Function (ACF) for Residuals for predicted Australian Residents')
plt.grid(True)
# There seems to be a sesonality dependence still on the residuals of the model



In [ ]:
# 1B another way with noise
# Add lagged values as a new column
australian_res_df2 = australian_res_df.copy()
australian_res_df2['Lagged'] = australian_res_df2['Australian Residents'].shift(1)

# Drop the first row with NaN value due to lagging
australian_res_df2.dropna(inplace=True)

# Define the predictors (including the lagged values and the 't' variable)
X = australian_res_df2[['Lagged']]
X['t'] = australian_res_df2.index
X = sm.add_constant(X)  # Add a constant term for the intercept

# Define the response variable
y = australian_res_df2['Australian Residents']

# Fit the OLS regression model
auto_reg_model_australian_res2 = sm.OLS(y, X).fit()

# Print the summary of the model
print(auto_reg_model_australian_res2.summary())

# Produce the ACF plot of the residuals
residuals_australian_res2 = auto_reg_model_australian_res2.resid
plt.figure(figsize=(10, 6))
plot_acf(residuals_australian_res2, lags=70)
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.title('Autocorrelation Function (ACF) for Residuals for predicted Australian Residents 2')
plt.grid(True)

In [ ]:
# 1C Import and view Data

# Load the airquality dataset from R
r_data3 = r('datasets::BJsales')

# Display the DataFrame
print(r_data3)

# Create a pandas DataFrame 
index3 = np.arange(len(r_data3))
bjsales_df = pd.DataFrame({
    't': index3,
    'BJ_Sales': r_data3
})
bjsales_df.set_index('t', inplace=True)
print(bjsales_df.describe())

# Count NA values
na_count = bjsales_df.isna().sum().sum()

# Count infinite values
inf_count = np.isinf(bjsales_df).sum().sum()
print(f"NA count: {na_count}, 'Infinity count': {inf_count}")

In [ ]:
# 1C
plt.figure(figsize=(10, 6))
plt.plot(bjsales_df, marker='o', linestyle='-')
plt.xlabel('T')
plt.ylabel('BJ Sales')
plt.title('BJ Sales')
plt.grid(True)
plt.show()
# The most prevailing characterisitic seams to autocorrelation

# Create the ACF plot
plt.figure(figsize=(10, 6))
plot_acf(bjsales_df, lags=149)
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.title('Autocorrelation Function (ACF) for BJ sales')
plt.grid(True)
plt.show()
# The autocorrelation plot shows a slow decay in the autocorrelation, indicating that the data could be a random walk

bjsales_df2 = bjsales_df.copy()
bjsales_df2['Lagged'] = bjsales_df2['BJ_Sales'].shift(1)

# Drop the first row with NaN value due to lagging
bjsales_df2.dropna(inplace=True)

X_bj = bjsales_df2[['Lagged']]
X_bj = sm.add_constant(X_bj)  # Add a constant term for the intercept

# Define the response variable
y_bj = bjsales_df2['BJ_Sales']

# Fit the OLS regression model
auto_reg_model_bj = sm.OLS(y_bj, X_bj).fit()

# Print the summary of the model
print(auto_reg_model_bj.summary())

# Produce the ACF plot of the residuals
auto_reg_model_bj_res = auto_reg_model_bj.resid
plt.figure(figsize=(10, 6))
plot_acf(auto_reg_model_bj_res, lags=148)
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.title('Autocorrelation Function (ACF) for Residuals for predicted BJ sales')
plt.grid(True)

#Residuals still seam to have a temporal dependence (seasonality)

In [ ]:
# Prob 2 Import and Data
import pandas as pd
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
import numpy as np


food = pd.read_csv("Food_Inspections_proc.csv")

# Create new columns and drop the specified columns
food = food.assign(
    FAIL=food['Results'] == "Fail",
    SERIOUS_FAIL=food['IsSerious'],
    PASS=food['Results'].isin(["Pass", "Pass w/ Conditions"]),
    NO_ENTRY=food['Results'] == "No Entry",
    NOT_READY=food['Results'] == "Not Ready"
).drop(columns=['Results', 'IsSerious'])

# Display the DataFrame
print(food.head())

In [ ]:
# 2A

# Group by 'DBA.Name' and calculate the number of inspections
inspection_counts = food.groupby('DBA.Name').size().reset_index(name='NUM')

# Filter for entries with at least 40 inspections
filtered_restaurants = food[food['DBA.Name'].isin(inspection_counts[inspection_counts['NUM'] >= 40]['DBA.Name'])]

# Group by 'DBA.Name' and calculate NUM and PASS_RATE
quality = filtered_restaurants.groupby('DBA.Name').agg(
    NUM=('DBA.Name', 'size'),
    PASS_RATE=('PASS', 'mean')
).reset_index()

# Sort the DataFrame by PASS_RATE in descending order
quality = quality.sort_values(by='PASS_RATE', ascending=False)

# Display the DataFrame
pd.set_option('display.max_rows', None)
print(quality)
pd.set_option('display.max_rows', 15)

# I Buy a looot of cooked food at Marianos. It's .81 pass rate, is decent but not great. I would expect it to be higher. I will be on the look out at what I buy and the quality of the food next time


In [ ]:
# 2B

# Group by 'Year' and 'Month' and calculate TESTS and PASS_RATE
agg = (
    food.groupby(['Year', 'Month'])
    .agg(
        TESTS=('PASS', 'count'),  # Count the number of rows in the 'PASS' column
        TOTAL_PASS=('PASS', 'sum'),  # Sum of 'PASS'
        TOTAL_FAIL=('FAIL', 'sum')  # Sum of 'FAIL'
    )
    .reset_index()  # Reset index to make 'Year' and 'Month' regular columns
)

# Calculate PASS_RATE after grouping and aggregation
agg['PASS_RATE'] = agg['TOTAL_PASS'] / (agg['TOTAL_PASS'] + agg['TOTAL_FAIL'])

# Add the DATE column
agg['DATE'] = agg['Year'].astype(str) + "-" + agg['Month'].astype(str)

# Drop unnecessary intermediate columns (if needed)
agg = agg.drop(columns=['TOTAL_PASS', 'TOTAL_FAIL'])

# Display the DataFrame
print(agg)
print(agg.describe())   

# Plot
pass_rate = agg['PASS_RATE']
plt.figure(figsize=(12, 6))
plt.plot(pass_rate, linestyle='-', marker='o')
# Set x-axis labels
plt.xticks(ticks=range(0, len(agg['DATE']), 2), labels=agg['DATE'][::2], rotation=290, ha='right')
# Set axis labels and title
plt.xlabel('Date')
plt.ylabel('Pass Rate')
plt.title('Pass Rate Over Time')
plt.grid(True)

# 143 Observations
# From 2010 to 2021 so 12 years
# Frequency is monthly (K=12)

In [ ]:
# 2C
# Create the ACF plot

plt.figure(figsize=(10, 6))
plot_acf(pass_rate, lags=142)
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.title('Autocorrelation Function (ACF) for Food Observations')
plt.grid(True)
plt.show()
# We do observe a strong autocorrelation in the data. It aooears to be stationary!

In [ ]:
# 2D
pass_rate2 = pass_rate.to_frame(name='PASS_RATE')
pass_rate2['Lagged'] = pass_rate2.shift(1)

# Drop the first row with NaN value due to lagging
pass_rate2.dropna(inplace=True)

X_agg = pass_rate2[['Lagged']]
X_agg = sm.add_constant(X_agg)  # Add a constant term for the intercept

# Define the response variable
y_agg = pass_rate2['PASS_RATE']

# Fit the OLS regression model
auto_reg_model_agg = sm.OLS(y_agg, X_agg).fit()

# Print the summary of the model
print(auto_reg_model_agg.summary())
b0 = auto_reg_model_agg.params[0]
b1 = auto_reg_model_agg.params[1]

print(f"The estimated mean level for the statioanry time series is {b0/(1-b1)}")

In [ ]:
# 2E
# Produce the ACF plot of the residuals

auto_reg_model_agg_res = auto_reg_model_agg.resid
plt.figure(figsize=(12, 6))
plot_acf(auto_reg_model_agg_res, lags=60)
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.title('Autocorrelation Function (ACF) for Residuals for predicted Agg Resturant Path Rate')
plt.grid(True)

#It appears that lag 12 has a strong autocorrelation

In [ ]:
# 2F-1 Add seasonal predictors (sine and cosine for periodicity)
n = len(pass_rate2)
time = np.arange(n)  # Time index
seasonality_order = 12  # yearly seasonality

pass_rate2['sin'] = np.sin(2 * np.pi * time / seasonality_order)
pass_rate2['cos'] = np.cos(2 * np.pi * time / seasonality_order)

# Redefine the predictors (X) with seasonality terms
X_seasonal = pass_rate2[['Lagged', 'sin', 'cos']]
X_seasonal = sm.add_constant(X_seasonal)

# Define the response variable
y_agg = pass_rate2['PASS_RATE']

# Fit the seasonal model
seasonal_model = sm.OLS(y_agg, X_seasonal).fit()
print(seasonal_model.summary())

print("--------------------")
print("AR(1) Model AIC:", auto_reg_model_agg.aic)
print("Seasonal Model AIC:", seasonal_model.aic)
# Determine which model has the lower AIC
if seasonal_model.aic < auto_reg_model_agg.aic:
    print("The seasonal model is better (lower AIC).")
else:
    print("The AR(1) model is better (lower AIC).")
print("--------------------")
anova_results = sm.stats.anova_lm(auto_reg_model_agg, seasonal_model)
print(anova_results)


In [ ]:
# 2F-2 Adding dummy variables for the months with vacations
# Assume "agg" contains the 'Month' information
pass_rate2['Month'] = agg['Month']  # Add the 'Month' column from agg to pass_rate2


# Create dummy variables for the categorical 'Month' variable
month_dummies = pd.get_dummies(pass_rate2['Month'], prefix='Month', drop_first=True)  # Drop the first category to avoid multicollinearity

# Ensure boolean month columns are converted to numeric (if necessary)
month_dummies = month_dummies.astype(int)  # Convert True/False to 1/0

# Combine the Lagged variable, sine/cosine, and month dummies into predictors
X_seasonal2 = pd.concat([pass_rate2[['Lagged']], month_dummies], axis=1)
X_seasonal2 = sm.add_constant(X_seasonal2)  # Add constant for intercept

# Define the response variable
y_agg = pass_rate2['PASS_RATE']

X_seasonal2 = X_seasonal2.loc[y_agg.index]

# Fit the seasonal model with the month categorical variables
seasonal_model_with_months = sm.OLS(y_agg, X_seasonal2).fit()
print(seasonal_model_with_months.summary())

In [166]:
# 2F-3 Just with mont h12
# Assume "agg" contains the 'Month' information
pass_rate2['Month_12'] = (pass_rate2['Month'] == 12).astype(int)  # True for December, else False (converted to 1/0)

# Combine predictors
X_seasonal3 = pass_rate2[['Lagged', 'Month_12']]
X_seasonal3 = sm.add_constant(X_seasonal3)  # Add constant for intercept

# Define the response variable
y_agg = pass_rate2['PASS_RATE']

# Fit the seasonal model with only Month 12 as a categorical predictor
seasonal_model_with_month12 = sm.OLS(y_agg, X_seasonal3).fit()

# Print the summary of the model
print(seasonal_model_with_month12.summary())
print("--------------------")
print("AR(1) Model AIC:", auto_reg_model_agg.aic)
print("Seasonal Model AIC:", seasonal_model.aic)
# Determine which model has the lower AIC
if seasonal_model_with_month12.aic < auto_reg_model_agg.aic:
    print("The seasonal model-month12 is better (lower AIC).")
else:
    print("The AR(1) model is better (lower AIC).")
print("--------------------")
anova_results = sm.stats.anova_lm(auto_reg_model_agg, seasonal_model_with_month12)
print(anova_results)

                            OLS Regression Results                            
Dep. Variable:              PASS_RATE   R-squared:                       0.522
Model:                            OLS   Adj. R-squared:                  0.515
Method:                 Least Squares   F-statistic:                     75.75
Date:                Wed, 27 Nov 2024   Prob (F-statistic):           5.63e-23
Time:                        13:03:48   Log-Likelihood:                 373.80
No. Observations:                 142   AIC:                            -741.6
Df Residuals:                     139   BIC:                            -732.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2604      0.046      5.715      0.0